# SVM ソフトマージン
### ホールドアウト
sklearn.model_selection.train_test_split http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html  
sklearn.metrics http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics  

matplotlibのカラーマップを使用 https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import   accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.datasets import make_classification

import matplotlib.pyplot as plt
###from matplotlib import cm # maplotlibのカラーマップを使用
%matplotlib inline

In [ ]:
X, y = make_classification( n_samples=100, n_features=2, n_informative=2, n_redundant=0, 
                            n_classes=2, n_clusters_per_class=1, 
                            class_sep=0.4, #　大きいほどクラス分離の距離が大きい 
                            shift=None, 
                            random_state=5) # 整数を与えると乱数の再現性がある

#### トレーニングデータとテストデータに分離（split）
sklearn.model_selection.train_test_split  
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

fig = plt.subplots(figsize=(6,4))
plt.scatter(X_train[:,0], X_train[:,1], c=y_train, edgecolor='k', cmap=plt.cm.gray)
#plt.savefig('fig_SVM_SM_Train.pdf')
plt.show()

In [ ]:
clf = svm.SVC(kernel='linear', C=10000) # Cの値を大きくして，なるべく他クラスへの混入を認めない
clf.fit(X_train, y_train)

In [ ]:
fig = plt.subplots(figsize=(6,4))

plt.scatter(X_train[:, 0], X_train[:, 1], s=30, c=y_train, edgecolor='k', cmap=plt.cm.gray)

# plot hyper-plane and margin liness as the lines
ax = plt.gca() # get current axis
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate the lines
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T

# 超平面までの距離を求め，次にグリッドに合わせてreshape(2次元)
Z = clf.decision_function(xy).reshape(XX.shape)

# マージンの境界線と超平面を等高線に描く
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, #等高線の本数と間隔をlevelsで指定
           linestyles=['--', '-', '--'])

plt.colorbar()

# plot support vectors
#ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=200, c='w', edgecolors='k')
#plt.savefig('fig_SVM_SM_Train_Result.pdf')

plt.show()

#### トレーニングデータに対する評価

In [ ]:
y_train[0:10]
print('トレーニングデータ　正解率', clf.score(X_train, y_train))   

In [ ]:
y_train_pred = clf.predict(X_train)
print("true value      : %s" % y_train)
print("prediction value: %s" % y_train_pred)
print('confusion = \n %s' % confusion_matrix(y_train, y_train_pred)) # 混同行列

トレーニングデータに対する評価，下記のような個別の評価も可能。

In [ ]:
print('accuracy = %f ' % accuracy_score(y_train, y_train_pred))  # 正答率
print('precision = %f ' % precision_score(y_train, y_train_pred)) # 適合率
print('recall = %f ' % recall_score(y_train, y_train_pred)) # 再現率
print('F-measure = %f' % f1_score(y_train, y_train_pred)) # F-値

上記は，次で一括して計算できる。

In [ ]:
print( classification_report(y_train, y_train_pred)) # 正答率（accuracy）はavg/totalに

上記の結果の見方，"0", "1"はクラス名で，それぞれの立場での 評価指標が示されている。
上記の一つ一つの評価では，"1"の立場で見ていることと同じである。また，F-measureとf1-scoreは同じ量を示す。

#### テストデータに対する評価

In [ ]:
y_test_pred = clf.predict(X_test)
print('true value      : %s' % y_test)
print('prediction value: %s' % y_test_pred)
print('confusion matrix =  %s' % confusion_matrix(y_test, y_test_pred))

In [ ]:
print('accuracy  = %f ' % accuracy_score(y_test, y_test_pred))
print('precision = %f ' % precision_score(y_test, y_test_pred))
print('recall    = %f ' % recall_score(y_test, y_test_pred))
print('F-measure = %f' % f1_score(y_test, y_test_pred))

#### 次の見方
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
- https://gotutiyan.hatenablog.com/entry/2020/09/09/111840

In [ ]:
print( classification_report(y_test, y_test_pred))

#### ダミーデータを用いたclassification_reportの結果を見る

In [ ]:
y_true = [0, 1, 2, 2, 2]
y_pred = [0, 0, 2, 2, 1]
print(classification_report(y_true, y_pred))

#### 上記の計算を検算する

In [ ]:
# precision
prec_0 = 1/2
prec_1 = 0/1
prec_2 = 2/2
prec = (prec_0+prec_1+prec_2)/3
prec

In [ ]:
#recall
rec_0 = 1/1
rec_1 = 0/1
rec_2 = 2/3
rec = (rec_0+rec_1+rec_2)/3
rec